In [1]:
using VCFTool

In [2]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_example_file_path = joinpath(input_dir, "example.vcf.gz")

"../input/example.vcf.gz"

In [3]:
vcf_738 = read_vcf(vcf_738_file_path)

vcf_example = read_vcf(vcf_example_file_path)

vcf_to_use = vcf_738

,CHROM,POS,ID,REF,ALT,QUAL,FILTER
,String,Int64,String,String,String,String,String
1,chr1,10439,.,AC,A,72,PASS
2,chr1,13284,.,G,A,60,LowGQX;NoPassedVariantGTs
3,chr1,13868,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
4,chr1,15274,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
5,chr1,15903,.,G,GC,209,PASS
6,chr1,16683,.,T,G,0,LowGQX;NoPassedVariantGTs
7,chr1,16713,.,T,G,0,LowGQX;LowDepth;NoPassedVariantGTs
8,chr1,16731,.,C,G,0,LowGQX;NoPassedVariantGTs
9,chr1,16734,.,T,G,4,LowGQX;NoPassedVariantGTs


In [4]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

"../input/cardiotoxicity.bed"

## Check BED

In [5]:
using DataFrames

using CSV: File

bed_io = open("../input/cardiotoxicity.bed")

bed = DataFrame(File(bed_io; delim='\t', header=false))

println(bed)

close(bed_io)

8×4 DataFrame
 Row │ Column1  Column2    Column3    Column4                          
     │ String   Int64      Int64      String                           
─────┼─────────────────────────────────────────────────────────────────
   1 │ chr2     240630710  240630710  rs12468485
   2 │ chr7     150999023  150999023  rs1799983
   3 │ chr10     99836239   99836239  rs17222723
   4 │ chr10     99851537   99851537  rs8187710
   5 │ chr12     53211761   53211761  rs2229774(childhood cancer risk)
   6 │ chr16     88646828   88646828  rs4673
   7 │ chr19     51354484   51354484  rs79338777
   8 │ chr21     36146408   36146408  rs1056892


## Tabix Regions

In [7]:
tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

LoadError: MethodError: no method matching print_and_run_cmd(::Cmd, ::String)
Closest candidates are:
  print_and_run_cmd(::Base.AbstractCmd) at /Users/kate/github/VCFTool.jl/src/print_and_run_cmd.jl:1

## Check Variants

https://cardiooncologyjournal.biomedcentral.com/articles/10.1186/s40959-020-00060-0/tables/3

In [ ]:
# rs1799983

using Query, DataFrames

x = @from i in vcf_to_use begin
    @where i.CHROM=="chr7"
    @where i.POS==150999023
    @select {i.CHROM, i.POS, i.REF, i.ALT, i.FORMAT, i.Germ}
    @collect DataFrame
end

println(x)